In [1]:
import sklearn
print(sklearn.__version__)

0.24.2


# Titanic Data Cleaning, Preprocessing, and Analysis
## ECS 171, Team 5
## July 2021

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib notebook -> will lead to interactive plots embedded within the notebook
#%matplotlib inline   -> will lead to static images of your plot embedded in the notebook
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [21]:
#load dataset
titanic = pd.read_csv("train.csv")

In [22]:
#output first five entries
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [23]:
#inspect data types, columns, number of entries and data usage
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [24]:
# there are 891 observations and 12 columns, one of which is the target variable Survived
titanic.shape

(891, 12)

In [25]:
#check for na values
titanic.isna().sum(axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Data Cleaning and Imputation

The three variables which have NA values are Age, Cabin Number, and Embarked.
* 177/891 or around 20% of Age values are NA, and 687/891 or approximately 77% of Cabin values are NA.
* The entries corresponding to the NA Embarked values likely can be removed since only 2/891 are missing.
* For the Age variable, however, we should impute the missing age values since these are an important predictor for survival.
* Since the majority of values in Cabin Number are missing, we will drop that column.

In [26]:
#copy the dataset to prevent any overwriting 
titanic_copy=titanic.copy() 
#drop the variable 'Cabin'
titanic_copy.drop(columns='Cabin',inplace=True)

In [27]:
#extract the rows with NA Embarked values and drop them from titanic_copy
titanic_copy[titanic_copy.Embarked.isna()]
titanic_copy=titanic_copy[~titanic_copy.Embarked.isna()]

In [28]:
#check the shape of the dataset now, it is 889 records and 11 columns
titanic_copy.shape

(889, 11)

In [29]:
#check to see that NAs other than the Age column have been dropped
titanic_copy.isna().sum(axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

Now let us perform data imputation on the Age column using KNNImputer from sklearn. 

In [30]:
#extract only the numeric columns from titanic dataset
titanic_nums=titanic.select_dtypes(include=np.number)
titanic_nums

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000
887,888,1,1,19.0,0,0,30.0000
888,889,0,3,NaN,1,2,23.4500
889,890,1,1,26.0,0,0,30.0000


In [31]:
#KNN Imputation with number of neighbors = 5, and the weights are uniformly averaged by measure of distances from each neighbor
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")
knn_imputer.fit_transform(titanic_nums) # fit & transform

array([[  1.    ,   0.    ,   3.    , ...,   1.    ,   0.    ,   7.25  ],
       [  2.    ,   1.    ,   1.    , ...,   1.    ,   0.    ,  71.2833],
       [  3.    ,   1.    ,   3.    , ...,   0.    ,   0.    ,   7.925 ],
       ...,
       [889.    ,   0.    ,   3.    , ...,   1.    ,   2.    ,  23.45  ],
       [890.    ,   1.    ,   1.    , ...,   0.    ,   0.    ,  30.    ],
       [891.    ,   0.    ,   3.    , ...,   0.    ,   0.    ,   7.75  ]])

In [32]:
#again we extract the numeric columns, this time from the titanic_copy dataset
titanic_copy_nums=titanic_copy.select_dtypes(include=np.number) 

#then we substitute our new imputated values for the missing values in titanic copy
titanic_copy[titanic_copy_nums.columns]=knn_imputer.transform(titanic_copy_nums)

After this imputation process, let us check if we have any NAs left:

In [43]:
titanic_copy.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

There are no NAs, so now let us see what has happened to our datatypes after the imputation:

In [40]:
titanic_copy.dtypes

PassengerId    float64
Survived       float64
Pclass         float64
Name            object
Sex             object
Age            float64
SibSp          float64
Parch          float64
Ticket          object
Fare           float64
Embarked        object
dtype: object

We can now see that our Survived column is of type float, which is not what we want as a binary value. Also PassengerId, Pclass, SibSp, and Parch were originally of integer values. We will convert all these back to int.

In [62]:
#convert all float versions of these variables back to int
titanic_copy.Survived=titanic_copy.Survived.astype('int')
titanic_copy.PassengerId=titanic_copy.PassengerId.astype('int')
titanic_copy.Pclass=titanic_copy.Pclass.astype('int')
titanic_copy.SibSp=titanic_copy.SibSp.astype('int')
titanic_copy.Parch=titanic_copy.Parch.astype('int')

## Final Version of Titanic Dataset (for now)

As we can see in this final version of titanic_copy, there are no missing/NA values, and all columns are of datatype of either `int32`, `float64`, or `object`. Now our next step would be to separate X feature attributes from y target attribute of **Survived**, and use machine learning techniques to predict whether a person has survived based on X attributes.

In [63]:
titanic_copy

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,27.2,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [65]:
#variable desciption and memory usage
titanic_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int32  
 1   Survived     889 non-null    int32  
 2   Pclass       889 non-null    int32  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int32  
 7   Parch        889 non-null    int32  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int32(5), object(4)
memory usage: 66.0+ KB


In [61]:
#just some extra summary statistics
titanic_copy.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,446.000000,0.382452,2.311586,29.648673,0.524184,0.382452,32.096681
std,256.998173,0.486260,0.834700,13.290522,1.103705,0.806761,49.697504
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,224.000000,0.000000,2.000000,21.600000,0.000000,0.000000,7.895800
50%,446.000000,0.000000,3.000000,29.000000,0.000000,0.000000,14.454200
75%,668.000000,1.000000,3.000000,36.200000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [66]:
#download this version of the titanic dataset
titanic_copy.to_csv('titanic.csv', sep = ',', index = False)

In [18]:
titanic = pd.read_csv("titanic.csv")

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
